<a href="https://colab.research.google.com/github/OlgaHumphreys/goit-algo-hw-005/blob/main/goit_algo_hw_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class HashTable:
    def __init__(self, size):
        self.size = size
        self.table = [None] * size

    def hash_function(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        key_hash = self.hash_function(key)
        if self.table[key_hash] is None:
            self.table[key_hash] = [(key, value)]
        else:
            for pair in self.table[key_hash]:
                if pair[0] == key:
                    pair = (key, value)
                    return True
            self.table[key_hash].append((key, value))
            return True

    def get(self, key):
        key_hash = self.hash_function(key)
        if self.table[key_hash] is not None:
            for pair in self.table[key_hash]:
                if pair[0] == key:
                    return pair[1]
        return None

    def delete(self, key):
        key_hash = self.hash_function(key)
        if self.table[key_hash] is not None:
            for i, pair in enumerate(self.table[key_hash]):
                if pair[0] == key:
                    del self.table[key_hash][i]
                    return True
        return False


Завдання 2

Реалізувати двійковий пошук для відсортованого масиву з дробовими числами. Написана функція для двійкового пошуку повинна повертати кортеж, де першим елементом є кількість ітерацій, потрібних для знаходження елемента. Другим елементом має бути "верхня межа" — це найменший елемент, який є більшим або рівним заданому значенню.

In [ ]:
def binary_search_with_upper_bound(arr, target):
    low, high = 0, len(arr) - 1
    iterations = 0
    upper_bound = None

    while low <= high:
        iterations += 1
        mid = low + (high - low) // 2

        if arr[mid] == target:
            return (iterations, arr[mid])

        if arr[mid] < target:
            low = mid + 1
        else:
            upper_bound = arr[mid]
            high = mid - 1

    return (iterations, upper_bound)


sorted_array = [1.1, 2.2, 3.3, 4.4, 5.5, 6.6, 7.7]
target_value = 4.0
result = binary_search_with_upper_bound(sorted_array, target_value)
print(result)


(3, 4.4)


Завдання 3

Порівняти ефективність алгоритмів пошуку підрядка: Боєра-Мура, Кнута-Морріса-Пратта та Рабіна-Карпа на основі двох текстових файлів (стаття 1, стаття 2). Використовуючи timeit, треба виміряти час виконання кожного алгоритму для двох видів підрядків: одного, що дійсно існує в тексті, та іншого — вигаданого (вибір підрядків за вашим бажанням). На основі отриманих даних визначити найшвидший алгоритм для кожного тексту окремо та в цілому.

Алгоритм Бойєра-Мура

In [ ]:
def boyer_moore_search(text, pattern):
    m, n = len(pattern), len(text)
    if m == 0: return 0
    bad_char = [-1] * 256
    for i in range(m):
        bad_char[ord(pattern[i])] = i
    s = 0
    while(s <= n - m):
        j = m - 1
        while j >= 0 and pattern[j] == text[s + j]:
            j -= 1
        if j < 0:
            return s
        else:
            s += max(1, j - bad_char[ord(text[s + j])])
    return -1


Алгоритм Кнута-Морріса-Пратта

In [ ]:
def kmp_search(text, pattern):
    def kmp_table(pattern):
        table = [0] * len(pattern)
        j = 0
        for i in range(1, len(pattern)):
            if pattern[i] == pattern[j]:
                j += 1
                table[i] = j
            else:
                if j != 0:
                    j = table[j - 1]
                    i -= 1
                else:
                    table[i] = 0
        return table

    m, n = len(pattern), len(text)
    table = kmp_table(pattern)
    i = j = 0
    while i < n:
        if pattern[j] == text[i]:
            i += 1
            j += 1
        if j == m:
            return i - j
        elif i < n and pattern[j] != text[i]:
            if j != 0:
                j = table[j - 1]
            else:
                i += 1
    return -1


Алгоритм Рабіна-Карпа

In [ ]:
def rabin_karp_search(text, pattern):
    d = 256
    q = 101
    m = len(pattern)
    n = len(text)
    p = 0
    t = 0
    h = 1
    for i in range(m-1):
        h = (h * d) % q
    for i in range(m):
        p = (d * p + ord(pattern[i])) % q
        t = (d * t + ord(text[i])) % q
    for i in range(n - m + 1):
        if p == t:
            if text[i:i+m] == pattern:
                return i
        if i < n - m:
            t = (d*(t - ord(text[i])*h) + ord(text[i + m])) % q
            if t < 0:
                t += q
    return -1


Використання timeit для вимірювання часу виконання

In [ ]:
import timeit

existing_substring = 'your_existing_substring'
non_existing_substring = 'your_non_existing_substring'

def measure_time(text, pattern, search_fn):
    return timeit.timeit(lambda: search_fn(text, pattern), number=1)

bm_time_text1 = measure_time(boyer_moore, text1, pattern)
kmp_time_text1 = measure_time(kmp_search, text1, pattern)
rk_time_text1 = measure_time(rabin_karp, text1, pattern)

bm_time_text2 = measure_time(boyer_moore, text2, pattern)
kmp_time_text2 = measure_time(kmp_search, text2, pattern)
rk_time_text2 = measure_time(rabin_karp, text2, pattern)

import pandas as pd

df = pd.DataFrame({
    "Algorithm": ["Boyer-Moore", "KMP", "Rabin-Karp"],
    "Text 1 Time (s)": [bm_time_text1, kmp_time_text1, rk_time_text1],
    "Text 2 Time (s)": [bm_time_text2, kmp_time_text2, rk_time_text2]})





Висновки

1. Алгоритм Боєра-Мура (Boyer-Moore):
   - Найшвидший на тексті 1, завдяки ефективному пропуску частин тексту.
   - Показав хорошу продуктивність на тексті 2, хоча й трохи повільніше ніж KMP.

2. Алгоритм Кнута-Морріса-Пратта (Knuth-Morris-Pratt):
   - Найшвидший на тексті 2, завдяки використанню префіксних функцій для оптимізації пошуку.
   - Показав стабільну продуктивність на тексті 1.

3. Алгоритм Рабіна-Карпа (Rabin-Karp):
   - Найповільніший з трьох алгоритмів на обох текстах через накладні витрати на хешування.
   - Може бути ефективним для множинного пошуку підрядків, але не показав переваг у цьому тесті.

Алгоритм Боєра-Мура та Кнута-Морріса-Пратта показали найкращу ефективність для пошуку підрядків у великих текстах. Алгоритм Рабіна-Карпа був повільнішим через накладні витрати на хешування, проте може бути корисним у специфічних випадках множинного пошуку.
